In [36]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [37]:
#class열 타깃으로 사용, 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [38]:
#훈련세트를 훈련세트와 검증세트로 나눈다
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [35]:
# 전체, 훈련, 검증, 테스트
print(train_input.shape, sub_input.shape, val_input.shape, test_input.shape)

(5197, 3) (4157, 3) (1040, 3) (1300, 3)


In [6]:
# 결정트리 (전처리 필요없음)
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 과대적합 => 검증세트 만드느라 훈련세트 줄어서 / 보통 많은 데이터를 훈련할수록 사용하기 좋은 모델

0.9971133028626413
0.864423076923077


In [7]:
# 교차 검증 / cross_validate는 기본적으로 5-폴드 교차검증 수행
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit_time, score_time : 모델 훈련 시간, 검증 시간
# 교차 검증의 최종 점수는 test_score(검증 폴드의 점수)의 평균값 

{'fit_time': array([0.01324296, 0.01187658, 0.01163363, 0.01109409, 0.01059151]), 'score_time': array([0.00186849, 0.00185132, 0.00169539, 0.00174189, 0.00167918]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 교차검증의 최종 점수
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 타깃 글래스의 비율에 따라 골고루 나눠주는 기능
# cross_validate는 훈련 세트를 섞어 폴드를 나누지 않는다. train_test_split으로 이미 섞었기 때문.
# 만약 교차검증시 섞으려면 분할기(spliter) 지정해야함
# cross_validate는 회귀모델 => KFold, 분류모델 => StratifiedKFold 사용
# 앞서 수행한 교차검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 만약 훈련세트 섞은 후 10-폴드 교차 검증 수행시 코드
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
# 그리드 서치
# 하이퍼 파라미터 탐색과 교차 검증을 한번에..!
# cv매개변수 기본값 = 5
# min_impurity_decrease값마다 5-폴드 교차 검증 수행 => 5*5 25개 모델 훈련
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
# n_jobs 병렬실행에 사용할 CPU코어 수 / -1이면 가능한 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
# dt는 하이퍼 파라미터로 만들어진 모델
# 25개 모델중 검증 점수가 가장 높은 모델의 매개변수 조합으로
# 전체 훈련세트에서 자동으로 다시 모델 훈련 => best_estimator_
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
# 그리드 서치로 찾은 최적의 매개변수 => best_params_
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수 => mean_test_score 
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 가장 큰 값
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [18]:
# 좀 더 복잡한 매개변수 탐색
# 노드 분할위한 불순도 최소 감소량, 결정 트리의 최대깊이, 노드를 나누기위한 최소 샘플 수
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 0.0001에서 시작하여 0.001까지 0.0001씩 증가 배열 (0.0001 ~ 0.0009까지) 9
          'max_depth': range(5, 20, 1), # 5에서 20까지 1씩 증가 / 15
          'min_samples_split': range(2, 100, 10) # 2에서 100까지 10씩증가 / 10
          }
# 매개변수로 수행할 교차 검증 횟수 9*15*10 => 1350 기본 5-폴드 => 6750

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [23]:
# 랜덤 서치 (그리드 서치 같이 매개변수 값의 목록 전달이 아닌 매개변수 값 샘플링 할 수 있는 확률 분포 객체 전달)
# 앞에서 매개변수 폭 0.0001로 한걸 랜덤으로~
from scipy.stats import uniform, randint
#주어진 범위에서 고르게 값 뽑기 => 균등 분포에서 샘플링
#randint 정숫값
#uniform 실숫값

In [24]:
rgen = randint(0, 10) # 0부터 9까지
rgen.rvs(10) # 10개 샘플링

array([9, 7, 7, 3, 4, 1, 0, 8, 8, 2])

In [25]:
#균등 분포 샘플링
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  92, 112,  95, 101, 100, 103, 102, 102,  96]))

In [26]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.12368237, 0.87358637, 0.83808788, 0.9286657 , 0.35081295,
       0.10241871, 0.57359309, 0.50857095, 0.48423617, 0.51971975])

In [27]:
#랜덤 서치 이용한 결정 트리 훈련
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [28]:
# 랜덤 서치 클래스
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter => 샘플링횟수
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25e7d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ca6bf25c6d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25e9e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25d5d0>},
                   random_state=42)

In [29]:
#최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [31]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [32]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25e7d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ca6bf25c6d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25e9e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ca6bf25d5d0>},
                   random_state=42)

In [33]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
